In [ ]:
# [Nur Colab] Diese Zellen müssen nur auf *Google Colab* ausgeführt werden und installieren Packete und Daten
!wget -q https://raw.githubusercontent.com/KI-Campus/AMALEA/master/requirements.txt && pip install --quiet -r requirements.txt
!wget --quiet "https://github.com/KI-Campus/AMALEA/releases/download/data/data.zip" && unzip -q data.zip

# 🔧 Setup: Computer Vision Libraries

import warnings
warnings.filterwarnings('ignore')

# Standard Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import os
from typing import List, Tuple, Union

# Computer Vision  
import cv2
from scipy import signal, ndimage
import skimage
from skimage import feature, filters, segmentation, measure

# Machine Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import applications, layers, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Interactive Widgets
from ipywidgets import interact, interactive, fixed, widgets
from IPython.display import display, HTML

# Streamlit (für Apps)
import streamlit as st

# Konfiguration
plt.style.use('seaborn-v0_8-darkgrid')
np.random.seed(42)
tf.random.set_seed(42)
figure_inches = 10

print("✅ Computer Vision Libraries geladen!")
print(f"📊 TensorFlow: {tf.__version__}")
print(f"🖼️ OpenCV: {cv2.__version__}")
print(f"🔬 Scikit-Image: {skimage.__version__}")

# GPU Check
if tf.config.list_physical_devices('GPU'):
    print("🚀 GPU verfügbar für Deep Learning!")
else:
    print("💻 CPU wird verwendet")

# 🖼️ 06.2 Computer Vision Anwendungen - Von Theorie zur Praxis

**Data Analytics & Big Data - Woche 6.2**  
*IU Internationale Hochschule*

---

## 🎯 Lernziele

Nach diesem Notebook können Sie:
- ✅ **Faltungsoperationen** mathematisch verstehen und implementieren
- ✅ **Verschiedene Filter** (Roberts, Sobel, etc.) anwenden  
- ✅ **Praktische CV-Anwendungen** entwickeln (Objekterkennung, Segmentierung)
- ✅ **OpenCV** für professionelle Computer Vision nutzen
- ✅ **Streamlit-App** für Bildklassifikation erstellen

---

## 🧮 Faltungsoperationen verstehen

**Convolution** ist das Herzstück von Computer Vision! Lassen Sie uns verstehen, wie es funktioniert:

## 🧮 Faltungsoperation (Convolution) verstehen

### 🤔 Was ist Convolution?

**Convolution** ist eine mathematische Operation, die zwei Funktionen miteinander "faltet". In Computer Vision:
- **Bild** (Input) ⊛ **Filter** (Kernel) = **Feature Map** (Output)

### 📐 Mathematische Definition

**Kontinuierlich:** 
$$x(t) \ast y(t) = \int_{-\infty}^{+\infty} x(t -\tau) y(\tau) d\tau$$

**Diskret (für Pixel):**
$$x_n \ast y_n = \sum_{i = -\infty}^{\infty} x_i \cdot y_{n-i}$$

### 🎯 Warum ist das wichtig?

1. **Feature Detection:** Filter erkennen Muster (Kanten, Texturen, etc.)
2. **Parameter Sharing:** Ein Filter wird über das ganze Bild angewendet  
3. **Translation Invariance:** Objekte werden überall im Bild erkannt

### 🎮 Interaktive Convolution Demonstration

Verstehen Sie Convolution durch interaktive Rechteck-Funktionen:

In [1]:
from ipywidgets import interact, interactive, fixed, interact_manual
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from scipy import signal
figure_inches = 10


In [ ]:
def convolution_demo(tau: float, width1: float, width2: float):
    """
    📊 Interaktive Convolution-Demonstration
    
    Parameter:
    - tau: Verschiebung der zweiten Funktion
    - width1: Breite der ersten Rechteck-Funktion
    - width2: Breite der zweiten Rechteck-Funktion
    """
    # Definiere x-Achse
    x1 = np.linspace(-3.5, 3.5, num=1000)
    dX = x1[1] - x1[0]    
    
    # Erstelle Rechteck-Funktionen
    rect1 = np.where(abs(x1) <= width1/2, 1, 0)  # rect₁(t)
    rect2 = np.where(abs(x1 - tau) <= width2/2, 1, 0)  # rect₂(t-τ)
    
    # Convolution berechnen
    conv = np.convolve(rect1, rect2, 'same') * dX 
    
    # Visualisierung
    plt.figure(figsize=(16.5, 4))
    
    # Plots
    plt.plot(x1, rect1, 'b', linewidth=2, label='📊 rect₁(t)')
    plt.plot(x1, rect2, 'r', linewidth=2, label='📊 rect₂(t-τ)')
    
    # Convolution bis zu aktueller Position
    x_gr = x1 - tau
    if tau <= 0:
        index = np.where((np.absolute(x_gr) - np.absolute(tau)) <= 0.004)
        index = index[0][0] if len(index[0]) > 0 else 0
    else:
        index = np.where(np.absolute(x_gr - tau) <= 0.004)
        index = index[0][0] if len(index[0]) > 0 else 999
        
    plt.plot(x_gr[:index], conv[:index], 'g', linewidth=3, 
             label='🎯 rect₁ ⊛ rect₂ (Convolution)')
    
    # Vertikale Linie bei aktueller Position
    plt.axvline(x=tau, color='r', linestyle='--', alpha=0.7, linewidth=2)
    
    # Styling
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., prop={'size': 13})
    plt.ylim(0, np.maximum(np.max(conv), np.max(rect1)) + 0.1)
    plt.xlim(-2.5, 2.5)
    plt.grid(True, alpha=0.3)
    plt.title(f'🎮 Convolution Demo: τ = {tau:.2f}', fontsize=14, fontweight='bold')
    plt.xlabel('Zeit t', fontsize=12)
    plt.ylabel('Amplitude', fontsize=12)
    
    plt.tight_layout()
    plt.show()
    
    # Info-Text
    overlap_area = np.trapz(conv[:index], x_gr[:index]) if index > 0 else 0
    print(f"📊 Convolution Wert bei τ={tau:.2f}: {overlap_area:.3f}")
    print(f"💡 Interpretation: Überlappungsbereich der beiden Rechtecke")

In [ ]:
# 🎮 Interaktive Convolution Demo

print("🎯 Convolution verstehen durch interaktive Rechteck-Funktionen:")
print("• Bewegen Sie τ (tau) um die zweite Funktion zu verschieben")
print("• Ändern Sie width1/width2 um die Rechteck-Breiten anzupassen")
print("• Beobachten Sie, wie sich die Convolution (grün) entwickelt")

# Widget für interaktive Convolution
interactive_plot = interactive(
    convolution_demo, 
    tau=widgets.FloatSlider(
        value=0, min=-2.0, max=2.0, step=0.1,
        description='τ (Verschiebung):',
        style={'description_width': 'initial'}
    ),
    width1=widgets.FloatSlider(
        value=1.0, min=0.25, max=1.75, step=0.25,
        description='Width 1:',
        style={'description_width': 'initial'}  
    ),
    width2=widgets.FloatSlider(
        value=1.0, min=0.25, max=1.75, step=0.25,
        description='Width 2:',
        style={'description_width': 'initial'}
    )
)

# Widget anzeigen
display(interactive_plot)

print("\n💡 Key Insight: Convolution misst die 'Ähnlichkeit' zwischen zwei Funktionen!")
print("🔗 In CNNs: Bild ⊛ Filter = Feature Map (zeigt wo Features gefunden wurden)")

Natürlich erfolgt die Berechnung der Faltung nicht im kontinuierlichen Bereich. Daher verwendet numpy die folgende Formel für diskrete Faltung im 1-dimensionalen Raum:

\begin{equation*}
x_n \ast y_n = \sum_{i = -\infty}^{\infty} x_i  \  y_{n-i} = \sum_{i = -\infty}^{\infty} y_{i}  \ x_{n-i}
\end{equation*}

## 🖼️ 2D-Convolution für Bilder

### 📐 Von 1D zu 2D

**2D-Convolution** für Bilder:
$$x_{mn} \star y_{mn} = \sum_{i} \sum_{j} x_{ij} \cdot y_{m-i, n-j}$$

### 🔧 Praktische Parameter

**Bildformat:** Height × Width × Channels (H × W × C)
- **Graustufenbild:** 28 × 28 × 1  
- **RGB-Bild:** 224 × 224 × 3

**Filter-Parameter:**
- **Kernel Size (K):** z.B. 3×3, 5×5  
- **Stride (S):** Schrittweite (meist 1 oder 2)
- **Padding (P):** Randbehandlung (SAME, VALID)

### 📏 Ausgabegröße berechnen

$$W_{out} = \frac{W_{in} - K + 2P}{S} + 1$$
$$H_{out} = \frac{H_{in} - K + 2P}{S} + 1$$

### 🎯 Beispiel-Berechnung

```python
# Input: 28×28×1 (MNIST)
# Filter: 3×3, Stride=1, Padding=0
W_out = (28 - 3 + 0) / 1 + 1 = 26
H_out = (28 - 3 + 0) / 1 + 1 = 26  
# Output: 26×26×1
```

### 🔍 Convolution Visualisierung

Die Animation zeigt, wie ein 3×3 Filter über ein Bild "gleitet":

![Convolution Animation](images/Faltung1.png)

**Schritt-für-Schritt:**
1. **Filter positionieren** auf Bildbereich
2. **Element-wise Multiplikation** zwischen Filter und Pixeln  
3. **Summe bilden** → ein Output-Pixel
4. **Filter verschieben** (Stride) und wiederholen

<div class="alert alert-block alert-success">
<b>Aufgabe 5.2.1:</b> 
Implementieren Sie die Funktion <code>conv</code> welche ein gegebenes Bild <code>image_data</code> mit einem gegebenenen Filter <code>filter_kern</code> filtert. Nehmen Sie an:

* Das Bild liegt entsprechend dem Beispiel (in der folgenden Zelle) als eine Liste von Listen vor
* Die Tiefe des Bildes ist 1
</div>

In [ ]:
from typing import List

# 🔧 Convolution-Funktion implementieren

def conv(image_data: List[List[int]], filter_kern: List[List[int]]) -> List[List[int]]:
    """
    🎯 2D-Convolution Implementation (Educational)
    
    Parameter:
    - image_data: Bild als 2D-Liste [[pixel, pixel, ...], [row2, ...], ...]
    - filter_kern: Filter als 2D-Liste [[w1, w2], [w3, w4], ...]
    
    Returns:
    - Gefilterte Bilddata als 2D-Liste
    
    💡 Hinweis: Das ist eine vereinfachte Educational-Version.
       In der Praxis nutzen wir OpenCV oder TensorFlow!
    """
    
    # Dimensionen ermitteln
    image_height = len(image_data)
    image_width = len(image_data[0])
    filter_height = len(filter_kern)
    filter_width = len(filter_kern[0])
    
    # Output-Dimensionen berechnen (ohne Padding)
    output_height = image_height - filter_height + 1
    output_width = image_width - filter_width + 1
    
    # Initialisiere Output
    result = []
    
    # Convolution durchführen
    for i in range(output_height):
        row = []
        for j in range(output_width):
            # Berechne Convolution für aktuelle Position
            conv_sum = 0
            for fi in range(filter_height):
                for fj in range(filter_width):
                    # Element-wise Multiplikation und Summation
                    pixel_value = image_data[i + fi][j + fj]
                    filter_value = filter_kern[fi][fj]
                    conv_sum += pixel_value * filter_value
            
            row.append(conv_sum)
        result.append(row)
    
    return result

print("✅ Convolution-Funktion implementiert!")
print("\n📚 Lernziel: Verstehen Sie, wie Convolution 'unter der Haube' funktioniert")
print("🚀 In der Praxis: Nutzen Sie cv2.filter2D() oder TensorFlow für Performance!")

In [ ]:
# 🧪 Test der Convolution-Implementierung

print("🧪 Test-Case: Convolution mit Edge-Detection Filter")

# Test-Daten
test_input_data = [
    [0, 0, 0, 0, 0], 
    [0, 1, 1, 1, 0], 
    [0, 0, 2, 0, 0], 
    [0, 3, 3, 3, 0], 
    [0, 0, 0, 0, 0], 
    [0, 0, 0, 0, 0]
]

test_filter = [
    [0, 0], 
    [-1, 1]
]

expected_result = [
    [1, 0, 0, -1],
    [0, 2, -2, 0],
    [3, 0, 0, -3], 
    [0, 0, 0, 0], 
    [0, 0, 0, 0]
]

print("📊 Input Bild:")
for i, row in enumerate(test_input_data):
    print(f"Row {i}: {row}")

print(f"\n🔧 Filter (Edge Detection):")
for i, row in enumerate(test_filter):
    print(f"Filter Row {i}: {row}")

# Test ausführen
found = conv(test_input_data, test_filter)

print(f"\n🎯 Erwartetes Ergebnis:")
for i, row in enumerate(expected_result):
    print(f"Expected Row {i}: {row}")

print(f"\n✅ Gefundenes Ergebnis:")
for i, row in enumerate(found):
    print(f"Found Row {i}: {row}")

# Verification
try:
    assert found == expected_result
    print("\n🎉 Test erfolgreich! Ihre Convolution-Implementation ist korrekt!")
except AssertionError:
    print("\n❌ Test fehlgeschlagen. Überprüfen Sie Ihre Implementation.")
    print("💡 Tipp: Stellen Sie sicher, dass Sie Element-wise Multiplikation und Summation korrekt implementiert haben.")

print("\n💡 Was passiert hier?")
print("• Filter [-1, 1] erkennt horizontale Übergänge (Links-Rechts Unterschiede)")
print("• Positive Werte: Übergang von dunkel zu hell")  
print("• Negative Werte: Übergang von hell zu dunkel")
print("• Zero Werte: Keine Änderung")

### Filtertypen

Bevor wir nun in Richtung praktische Anwendung gehen, schauen wir uns grundlegende Filter an. Außerdem werden wir uns die Effekte der Filter anschauen - hierzu verwenden wir das folgende Bild:

In [ ]:
def read_image_as_array(image_path: str, new_size: Tuple[int, int] = (500, 500)) -> np.ndarray:
    """
    📸 Bild laden und als NumPy Array zurückgeben
    
    Parameter:
    - image_path: Pfad zur Bilddatei
    - new_size: Neue Bildgröße (width, height)
    
    Returns:
    - NumPy Array mit Graustufenwerten
    """
    try:
        img = Image.open(image_path).convert('L')  # Graustufen
        img = img.resize(new_size, Image.Resampling.LANCZOS)  # Moderne Resampling-Methode
        return np.array(img)
    except FileNotFoundError:
        print(f"❌ Bild nicht gefunden: {image_path}")
        # Fallback: Erstelle Beispielbild
        return create_sample_image(new_size)

def create_sample_image(size: Tuple[int, int] = (500, 500)) -> np.ndarray:
    """
    🎨 Erstellt ein Beispielbild mit geometrischen Formen
    """
    img = np.zeros(size)
    h, w = size
    
    # Rechteck
    img[h//4:3*h//4, w//4:w//2] = 255
    
    # Kreis  
    center = (3*w//4, h//2)
    y, x = np.ogrid[:h, :w]
    mask = (x - center[0])**2 + (y - center[1])**2 <= (w//8)**2
    img[mask] = 128
    
    return img

# Lade Testbild
print("📸 Lade Testbild...")

try:
    # Versuche Lama-Bild zu laden
    if os.path.exists('images/lama.png'):
        lama_array = read_image_as_array('images/lama.png', (500, 500))
        print("✅ Lama-Bild geladen!")
    else:
        print("⚠️  Lama-Bild nicht gefunden, erstelle Beispielbild...")
        lama_array = create_sample_image((500, 500))
        
    # Visualisierung
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Original Bild
    axes[0].imshow(lama_array, cmap='gray', interpolation='nearest')
    axes[0].set_title('🖼️ Testbild für Filter-Experimente', fontsize=14, fontweight='bold')
    axes[0].axis('off')
    
    # Histogram
    axes[1].hist(lama_array.flatten(), bins=50, alpha=0.7, color='skyblue', edgecolor='black')
    axes[1].set_title('📊 Pixel-Intensitäts-Verteilung', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Grauwert (0-255)')
    axes[1].set_ylabel('Anzahl Pixel')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"📏 Bildgröße: {lama_array.shape}")
    print(f"📊 Grauwert-Bereich: {lama_array.min()} - {lama_array.max()}")
    print(f"🎯 Durchschnitt: {lama_array.mean():.1f}")

except Exception as e:
    print(f"❌ Fehler beim Bildladen: {e}")
    lama_array = create_sample_image()
    print("🎨 Verwende generiertes Beispielbild")

#### Identitätsfilter

Der erste Filter entspricht der Identität, d.h. der Wert eines Pixel wird auf genau diesen abgebildet. Um dies zu erreichen wird ein quadratischer Filterkernel benötigt, dessen Größe ungerade ist. Außerdem ist der mittlere Eintrag 1 und alle anderen 0. Ein $3\times 3$-Filterkernel hat somit die Form:

$\left\lbrack\begin{array}{ccc} 0&0&0\\ 0&1&0\\ 0&0&0\end{array}\right\rbrack$

Und nun die angekündigte Anwendung auf das Bild:

In [ ]:
plt.figure(figsize=(figure_inches, figure_inches))
# Identitätsfilter definieren
identity_filter = [
    [0, 0, 0],
    [0, 1, 0], 
    [0, 0, 0]
]

# Filter anwenden (mit unserer Implementation)
filtered_identity = conv(data, identity_filter)

# Visualisierung
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

images_data = [
    (data, "🖼️ Original", "Original Bild"),
    (np.array(filtered_identity), "🔄 Identitätsfilter", "Sollte identisch sein"),
    (np.abs(data[1:-1, 1:-1] - np.array(filtered_identity)), "📊 Differenz", "Unterschied (sollte ≈0 sein)")
]

for i, (img, title, subtitle) in enumerate(images_data):
    axes[i].imshow(img, cmap='gray', interpolation='nearest')
    axes[i].set_title(title, fontsize=14, fontweight='bold')
    axes[i].text(0.02, 0.98, subtitle, transform=axes[i].transAxes, 
                bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8),
                verticalalignment='top', fontsize=10)
    axes[i].axis('off')

plt.tight_layout()
plt.show()

# Verifikation
original_roi = data[1:-1, 1:-1]  # Region of Interest (ohne Rand)
filtered_array = np.array(filtered_identity)
difference = np.abs(original_roi - filtered_array)
max_diff = np.max(difference)

print(f"\n📊 Verifikation:")
print(f"   Original ROI Shape: {original_roi.shape}")
print(f"   Filtered Shape: {filtered_array.shape}")
print(f"   Max Differenz: {max_diff}")

if max_diff < 1e-10:
    print("   ✅ Perfect! Identitätsfilter verändert nichts")
else:
    print(f"   ⚠️  Kleine Differenz gefunden (erwartet bei Integer-Implementierung)")

print("\n💡 Key Insight:")
print("• Identitätsfilter = Kernel mit 1 in der Mitte, 0 überall sonst")
print("• Wichtig für CNN-Architekturen (Skip Connections, ResNet)")
print("• Baseline um andere Filter zu verstehen")

#### Eckendetektoren

Die nächsten drei Filter ziehlen darauf ab, Ecken im Bild zu finden. Ziel hierbei ist es flächige Bereiche voneinander zu trennen. Die Filter sind oft nach deren Erfinder bzw. Entdecker benannt. In diesem Fall stellt der Sobel2 eine Verbesserung des Sobel1 dar - dieser kann zusätzlich zum horizontalen sowie vertikalen auch im $45^\circ$ Bereich messen.

Roberts: $\left\lbrack\begin{array}{ccc} 1&0&-1\\ 0&0&0\\ -1&0&1 \end{array}\right\rbrack$

Sobel1: $\left\lbrack\begin{array}{ccc} 0&-1&0\\ -1&4&-1\\ 0&-1&0\end{array}\right\rbrack$

Sobel2: $\left\lbrack\begin{array}{ccc} -1&-1&-1\\-1&8&-1\\ -1&-1&-1\end{array}\right\rbrack$

In [ ]:
# ⚡ Edge Detection Filter - Finde Kanten und Konturen!

print("🎯 Edge Detection: Findet Übergänge zwischen verschiedenen Bereichen")
print("💡 Anwendung: Objekterkennung, Contouring, Feature Extraction")

# Edge Detection Filter definieren
filters_edge = {
    "Roberts": {
        "kernel": [[1, 0, -1], [0, 0, 0], [-1, 0, 1]],
        "description": "Einfache Diagonal-Edge Detection"
    },
    "Sobel (Light)": {
        "kernel": [[0, -1, 0], [-1, 4, -1], [0, -1, 0]], 
        "description": "Kantenverstärkung in 4 Richtungen"
    },
    "Sobel (Strong)": {
        "kernel": [[-1, -1, -1], [-1, 8, -1], [-1, -1, -1]],
        "description": "Starke Kantenverstärkung in 8 Richtungen"
    }
}

# Moderne OpenCV Vergleiche
sobel_x_cv = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
sobel_y_cv = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])

print("\n🔧 Filter-Übersicht:")
fig, axes = plt.subplots(1, len(filters_edge), figsize=(15, 4))

for i, (name, filter_info) in enumerate(filters_edge.items()):
    kernel = np.array(filter_info["kernel"])
    
    # Filter visualisieren
    im = axes[i].imshow(kernel, cmap='RdBu', vmin=-8, vmax=8)
    axes[i].set_title(f'{name}', fontsize=12, fontweight='bold')
    
    # Werte in Zellen schreiben
    for (j, k), val in np.ndenumerate(kernel):
        axes[i].text(k, j, f'{val}', ha='center', va='center', 
                    color='white' if abs(val) > 2 else 'black', fontweight='bold')
    
    axes[i].set_xticks([])
    axes[i].set_yticks([])
    
    # Beschreibung
    axes[i].text(0.5, -0.15, filter_info["description"], 
                transform=axes[i].transAxes, ha='center', 
                fontsize=10, style='italic')

plt.tight_layout()
plt.show()

# Filter anwenden und vergleichen
print("\n⚙️  Anwenden der Edge-Detection Filter...")

# Roberts Filter anwenden
roberts_result = conv(lama_array.tolist(), filters_edge["Roberts"]["kernel"])

# Visualisierung der Ergebnisse
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Original
axes[0, 0].imshow(lama_array, cmap='gray')
axes[0, 0].set_title('🖼️ Original Bild', fontsize=14, fontweight='bold')
axes[0, 0].axis('off')

# Roberts Result
roberts_array = np.array(roberts_result)
axes[0, 1].imshow(np.abs(roberts_array), cmap='gray')  # Abs für bessere Sichtbarkeit
axes[0, 1].set_title('⚡ Roberts Edge Detection', fontsize=14, fontweight='bold')
axes[0, 1].axis('off')

# OpenCV Sobel X
sobel_x_result = cv2.filter2D(lama_array.astype(np.float32), -1, sobel_x_cv)
axes[1, 0].imshow(np.abs(sobel_x_result), cmap='gray')
axes[1, 0].set_title('🔍 Sobel X (OpenCV)', fontsize=14, fontweight='bold')
axes[1, 0].axis('off')

# OpenCV Sobel Y  
sobel_y_result = cv2.filter2D(lama_array.astype(np.float32), -1, sobel_y_cv)
axes[1, 1].imshow(np.abs(sobel_y_result), cmap='gray')
axes[1, 1].set_title('🔍 Sobel Y (OpenCV)', fontsize=14, fontweight='bold')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

# Statistiken
print(f"\n📊 Edge Detection Statistiken:")
print(f"   Roberts - Min: {roberts_array.min():.1f}, Max: {roberts_array.max():.1f}")
print(f"   Sobel X - Min: {sobel_x_result.min():.1f}, Max: {sobel_x_result.max():.1f}")
print(f"   Sobel Y - Min: {sobel_y_result.min():.1f}, Max: {sobel_y_result.max():.1f}")

print("\n💡 Edge Detection Insights:")
print("• Negative Werte = Übergang von hell zu dunkel")
print("• Positive Werte = Übergang von dunkel zu hell") 
print("• Absolutwerte = Kantenintensität (unabhängig von Richtung)")
print("• Sobel X = Vertikale Kanten, Sobel Y = Horizontale Kanten")

In [9]:
filter_kern_sobel1 = [[0,-1,0], [-1,4,-1], [0,-1,0]]

plt.figure(figsize=(figure_inches, figure_inches))
data = read_image_as_array('images/lama.png', (500,500))
filtered_data = conv(data, filter_kern_sobel1)

plt.imshow(filtered_data, cmap='gray')

---

## 🚀 Moderne Computer Vision mit OpenCV

### 💡 Von manuellen Filtern zu intelligenten Algorithmen

Bisher haben wir **manuelle Filter** kennengelernt. Jetzt schauen wir uns **intelligente CV-Algorithmen** an:

In [ ]:
# 🎯 Moderne Computer Vision Anwendungen mit OpenCV

print("🚀 Von manuellen Filtern zu intelligenten Algorithmen")

# 1. 📊 CANNY EDGE DETECTION - State-of-the-Art
def canny_edge_detection(image, low_threshold=50, high_threshold=150):
    """
    🔍 Canny Edge Detection - Beste Edge Detection Methode
    """
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) if len(image.shape) == 3 else image
    edges = cv2.Canny(gray, low_threshold, high_threshold)
    return edges

# 2. 🎯 CONTOUR DETECTION - Objektumrisse finden
def detect_contours(image):
    """
    🎯 Findet Objekt-Konturen im Bild
    """
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) if len(image.shape) == 3 else image
    
    # Canny edges
    edges = cv2.Canny(gray, 50, 150)
    
    # Contours finden
    contours, hierarchy = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Konturen auf Original zeichnen
    result = image.copy() if len(image.shape) == 3 else cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    cv2.drawContours(result, contours, -1, (0, 255, 0), 2)
    
    return result, contours

# 3. 🧩 IMAGE SEGMENTATION - K-Means Clustering
def kmeans_segmentation(image, k=4):
    """
    🧩 K-Means Bildsegmentierung
    """
    # Reshape für K-Means
    data = image.reshape((-1, 3))
    data = np.float32(data)
    
    # K-Means Parameter
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 20, 1.0)
    
    # K-Means ausführen
    _, labels, centers = cv2.kmeans(data, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    
    # Ergebnis rekonstruieren
    centers = np.uint8(centers)
    segmented_data = centers[labels.flatten()]
    segmented_image = segmented_data.reshape(image.shape)
    
    return segmented_image

# 4. 🔍 FEATURE DETECTION - SIFT Features
def detect_sift_features(image):
    """
    🔍 SIFT (Scale-Invariant Feature Transform) Features
    """
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) if len(image.shape) == 3 else image
    
    # SIFT Detector
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    
    # Keypoints zeichnen
    result = cv2.drawKeypoints(image, keypoints, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    
    return result, keypoints, descriptors

print("✅ Computer Vision Funktionen definiert!")

# Teste mit unserem Bild
if 'lama_array' in locals():
    # Konvertiere zu RGB falls nötig
    if len(lama_array.shape) == 2:
        test_image_rgb = cv2.cvtColor(lama_array, cv2.COLOR_GRAY2RGB)
    else:
        test_image_rgb = lama_array
        
    print(f"📸 Test-Bild bereit: {test_image_rgb.shape}")
else:
    print("⚠️  Laden Sie zuerst das Test-Bild!")

In [ ]:
# 🎮 Computer Vision Applications Demo

print("🎯 Anwendung aller CV-Algorithmen auf unser Test-Bild:")

# Stelle sicher, dass wir ein RGB-Bild haben
if len(lama_array.shape) == 2:
    demo_image = cv2.cvtColor(lama_array, cv2.COLOR_GRAY2RGB)
else:
    demo_image = lama_array.copy()

# 1. Canny Edge Detection
edges = canny_edge_detection(demo_image)

# 2. Contour Detection
contour_result, contours = detect_contours(demo_image)

# 3. K-Means Segmentation
segmented = kmeans_segmentation(demo_image, k=4)

# 4. SIFT Feature Detection
sift_result, keypoints, descriptors = detect_sift_features(demo_image)

# Große Visualisierung aller Ergebnisse
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Original
axes[0, 0].imshow(demo_image)
axes[0, 0].set_title('🖼️ Original Bild', fontsize=14, fontweight='bold')
axes[0, 0].axis('off')

# Canny Edges
axes[0, 1].imshow(edges, cmap='gray')
axes[0, 1].set_title('⚡ Canny Edge Detection', fontsize=14, fontweight='bold')
axes[0, 1].axis('off')

# Contours
axes[0, 2].imshow(contour_result)
axes[0, 2].set_title(f'🎯 Contours ({len(contours)} gefunden)', fontsize=14, fontweight='bold')
axes[0, 2].axis('off')

# K-Means Segmentation
axes[1, 0].imshow(segmented)
axes[1, 0].set_title('🧩 K-Means Segmentation', fontsize=14, fontweight='bold')
axes[1, 0].axis('off')

# SIFT Features
axes[1, 1].imshow(sift_result)
axes[1, 1].set_title(f'🔍 SIFT Features ({len(keypoints)} gefunden)', fontsize=14, fontweight='bold')
axes[1, 1].axis('off')

# Kombiniertes Ergebnis
combined = demo_image.copy()
# Canny Edges als rote Overlay
combined[edges > 0] = [255, 0, 0]
axes[1, 2].imshow(combined)
axes[1, 2].set_title('🎨 Kombiniert: Original + Edges', fontsize=14, fontweight='bold')
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

# Statistiken ausgeben
print(f"\n📊 Computer Vision Analyse-Ergebnisse:")
print(f"   🖼️  Bildgröße: {demo_image.shape}")
print(f"   ⚡ Canny Edges: {np.sum(edges > 0):,} Pixel ({np.sum(edges > 0)/edges.size*100:.1f}%)")
print(f"   🎯 Konturen: {len(contours)} Objekte erkannt")
print(f"   🔍 SIFT Features: {len(keypoints)} charakteristische Punkte")
print(f"   🧩 Segmentierung: 4 Bereiche unterschieden")

# Größte Kontur analysieren
if contours:
    largest_contour = max(contours, key=cv2.contourArea)
    contour_area = cv2.contourArea(largest_contour)
    image_area = demo_image.shape[0] * demo_image.shape[1]
    print(f"   📐 Größtes Objekt: {contour_area:.0f} Pixel ({contour_area/image_area*100:.1f}% des Bildes)")

print(f"\n💡 SIFT Descriptor Info:")
if descriptors is not None:
    print(f"   📊 Descriptor Matrix: {descriptors.shape}")
    print(f"   🔢 Pro Feature: {descriptors.shape[1]} Dimensionen")
    print(f"   💾 Speicher: {descriptors.nbytes:,} Bytes")

---

## 🎮 Interaktive Computer Vision Experimente

### 💻 Jupyter Widgets für Live-Demos

Testen Sie verschiedene CV-Parameter interaktiv:

In [ ]:
# 🎮 Interaktive Canny Edge Detection

def interactive_canny(low_threshold=50, high_threshold=150, blur_kernel=1):
    """
    🎯 Interaktive Canny Edge Detection mit Parameter-Tuning
    """
    # Bild vorbereiten
    img = demo_image.copy()
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    # Optional: Blur anwenden
    if blur_kernel > 1:
        gray = cv2.GaussianBlur(gray, (blur_kernel, blur_kernel), 0)
    
    # Canny Edge Detection
    edges = cv2.Canny(gray, low_threshold, high_threshold)
    
    # Visualisierung
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    # Original
    axes[0].imshow(img)
    axes[0].set_title('🖼️ Original', fontsize=14, fontweight='bold')
    axes[0].axis('off')
    
    # Graustufen (mit Blur)
    axes[1].imshow(gray, cmap='gray')
    title = f'📊 Graustufen' + (f' + Blur({blur_kernel}x{blur_kernel})' if blur_kernel > 1 else '')
    axes[1].set_title(title, fontsize=14, fontweight='bold')
    axes[1].axis('off')
    
    # Canny Edges
    axes[2].imshow(edges, cmap='gray')
    axes[2].set_title(f'⚡ Canny (L:{low_threshold}, H:{high_threshold})', fontsize=14, fontweight='bold')
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Statistiken
    edge_count = np.sum(edges > 0)
    edge_percentage = (edge_count / edges.size) * 100
    
    print(f"📊 Canny Statistiken:")
    print(f"   Threshold: Low={low_threshold}, High={high_threshold}")
    print(f"   Edge Pixel: {edge_count:,} ({edge_percentage:.2f}%)")
    print(f"   Blur Kernel: {blur_kernel}x{blur_kernel}")

# Widget erstellen
print("🎮 Interaktive Canny Edge Detection:")
print("• Low Threshold: Minimaler Gradient für Edge-Kandidaten")
print("• High Threshold: Minimaler Gradient für sichere Edges")  
print("• Blur Kernel: Rauschunterdrückung (ungerade Zahlen)")

interactive_canny_widget = interact(
    interactive_canny,
    low_threshold=widgets.IntSlider(
        value=50, min=10, max=200, step=10,
        description='Low Threshold:',
        style={'description_width': 'initial'}
    ),
    high_threshold=widgets.IntSlider(
        value=150, min=50, max=300, step=10,
        description='High Threshold:',
        style={'description_width': 'initial'}
    ),
    blur_kernel=widgets.IntSlider(
        value=1, min=1, max=15, step=2,
        description='Blur Kernel:',
        style={'description_width': 'initial'}
    )
)

print("\n💡 Tuning-Tipps:")
print("• Niedrige Thresholds = mehr Edges (mehr Rauschen)")
print("• Hohe Thresholds = weniger Edges (nur starke Kanten)")
print("• Blur reduziert Rauschen, kann aber Details verwischen")

---

## 🎮 Streamlit Computer Vision App

### 💻 Professionelle CV-Anwendung erstellen

Sie können eine **vollständige Computer Vision App** mit Streamlit erstellen:

```bash
# Im Terminal ausführen:
cd 06_Computer_Vision_NLP
streamlit run 06_02_streamlit_cv_apps.py
```

### 🌟 Features der CV-App:
- ✅ **Multiple CV-Algorithmen:** Edge Detection, Object Detection, Segmentation
- ✅ **Eigene Bilder hochladen** und analysieren
- ✅ **Parameter-Tuning** in Echtzeit
- ✅ **Feature Detection** (SIFT, ORB, Harris Corners)
- ✅ **Filter-Vergleiche** side-by-side

---

## 📚 Zusammenfassung & Key Takeaways

### ✅ Was Sie gelernt haben:

#### 1. 🧮 **Mathematische Grundlagen**
- **Convolution Operation:** Wie Filter mathematisch funktionieren
- **2D-Convolution:** Anwendung auf Bilder (H×W×C)
- **Parameter-Berechnung:** Kernel Size, Stride, Padding
- **Eigene Implementation:** Convolution von Grund auf verstehen

#### 2. 🔧 **Klassische Filter**
- **Identitätsfilter:** Baseline und Verständnis
- **Edge Detection:** Roberts, Sobel (Light/Strong)
- **Filter-Comparison:** Manuelle vs. OpenCV Implementation
- **Parameter-Tuning:** Optimale Einstellungen finden

#### 3. 🚀 **Moderne Computer Vision**
- **Canny Edge Detection:** State-of-the-Art Kantendetection
- **Contour Detection:** Objektumrisse automatisch finden
- **Image Segmentation:** K-Means Clustering für Bereiche
- **Feature Detection:** SIFT für charakteristische Punkte

#### 4. 🎮 **Praktische Anwendungen**
- **OpenCV Integration:** Professionelle CV-Library nutzen
- **Interactive Widgets:** Parameter live anpassen
- **Streamlit Apps:** Portfolio-ready CV-Anwendungen
- **Real-world Workflows:** Von Preprocessing bis Analyse

### 💡 **Von Theorie zur Praxis:**

```python
# Klassische Implementierung (Educational)
def conv(image, filter_kernel):
    # Manual convolution for understanding
    
# Moderne Implementierung (Production)
edges = cv2.Canny(image, 50, 150)
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
```

### 🎯 **Nächste Schritte:**

In den kommenden Notebooks vertiefen wir:
- **06.3:** Data Augmentation für robuste Modelle
- **06.4:** Transfer Learning mit vortrainierten CNNs
- **Portfolio:** Eigene CV-Projekte für Bewerbungen

---

### 🏆 **Portfolio-Projekt Challenge:**

1. **📸 Eigenes Bild-Dataset** sammeln (Selfies, Objekte, etc.)
2. **🔧 CV-Pipeline** entwickeln (Preprocessing → Detection → Analysis)
3. **🎮 Streamlit-App** erstellen mit Upload-Funktionalität
4. **📊 Analyse-Dashboard** mit Statistiken und Visualisierungen
5. **🚀 GitHub Repository** für Ihr Portfolio

**💪 Advanced Challenge:** Kombinieren Sie Edge Detection + Contour Detection + Feature Detection für ein intelligentes Objekt-Analyse-Tool!

In [10]:
filter_kern_sobel2 = [[-1,-1,-1], [-1,8,-1], [-1,-1,-1]]

plt.figure(figsize=(figure_inches, figure_inches))
data = read_image_as_array('images/lama.png', (500,500))
filtered_data = conv(data, filter_kern_sobel2)

plt.imshow(filtered_data, cmap='gray')

#### Bildschärfen

Der nächste Filter dient, wie der Name bereits vermuten lässt, dazu, dass Konturen im Bild schärfer werden.

$\left\lbrack\begin{array}{ccc} 0&-1&0\\ -1&5&-1\\ 0&-1&0 \end{array}\right\rbrack$

In [11]:
filter_kern_sharp = [[0,-1,0], [-1,5,-1], [0,-1,0]]

plt.figure(figsize=(figure_inches, figure_inches))
data = read_image_as_array('images/lama.png', (500,500))
filtered_data = conv(data, filter_kern_sharp)

plt.imshow(filtered_data, cmap='gray')

#### Blur / Unschärfe

Die letzen beiden Filter dienen dazu, das Bild zu glätten. Der erste Filter wird auch als Box-Linear-Filter bezeichnet und ist verhätlinismäßig relativ simple aufgebaut. Der zweite Filter basiert auf einer Gaußverteilung und wird daher als Gauß-Filter bezeichnet.

Box-Linear-Filter: $\frac{1}{9} \left\lbrack\begin{array}{ccc}1&1&1\\ 1&1&1\\ 1&1&1\end{array}\right\rbrack$

Gauß-Filter: $\frac{1}{16} \left\lbrack\begin{array}{ccc}1&2&1\\ 2&4&2\\ 1&2&1\end{array}\right\rbrack$

In [12]:
filter_kern_blf = [[1/9, 1/9, 1/9], [1/9, 1/9, 1/9], [1/9, 1/9, 1/9]]

plt.figure(figsize=(figure_inches, figure_inches))
data = read_image_as_array('images/lama.png', (500,500))
filtered_data = conv(data, filter_kern_blf)

plt.imshow(filtered_data, cmap='gray')

In [13]:
filter_kern_gauss = [[1/16, 2/16, 1/16], [2/16, 4/16, 2/16], [1/16, 2/16, 1/16]]

plt.figure(figsize=(figure_inches, figure_inches))
data = read_image_as_array('images/lama.png', (500,500))
filtered_data = conv(data, filter_kern_gauss)

plt.imshow(filtered_data, cmap='gray')

### RGB-Bilder

Farbige Bilder können in der Regel durch RGB-Bilder dargestellt werden, wobei $d$ gleich 3 ist und enthält:

- R (rot), 
- G (grün),
- B (blau)

Werte für alle Pixel in einem Bild.

In [14]:
lama = Image.open('images/lama.png')
lama = np.array(lama)

fig, ax = plt.subplots(figsize=(figure_inches, figure_inches))
ax.set_title('Lama image 768x1024', fontsize = 15)
ax.imshow(lama, interpolation='nearest')
plt.tight_layout()

In [15]:
# In general deep learning (and in tensorflow) Conv-layers will 
# regard all channels and therefore use "cubic" filter

# The filter used here in the example down below is only using d=1 (two - dimensional) of the 
# rgb image (therefore red), you can change [:,:,0] to [:,:,1] (green) and [:,:,2] (blue)!
# Try it! :)

prewitt_x =  np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])

lama_x_prew = signal.convolve2d(lama[:,:,0], prewitt_x, boundary='symm', mode='same')
lama_x_prew = np.absolute(lama_x_prew)

fig, ax = plt.subplots(figsize=(figure_inches, figure_inches))
ax.set_title('Horizontale Ableitung des Lama Bildes', fontsize = 15)
ax.imshow(lama_x_prew, interpolation='nearest', cmap='gray')
plt.tight_layout()

#### Die Faltungsschicht (engl. Convolutional Layer)

<img src="images/featuremaps.png" alt="Drawing" style="width: 600px;"/>

Eine Faltungsschicht, welche die erste Schicht im Netzwerk sein könnte, ist im Bild oben dargestellt. Ihr Kernel oder Filter mit den Dimensionen $K_x \times K_y \times d$ enthält Gewichte, die während des Trainings aktualisiert werden und auch die Darstellung der Bilder verändern. Eine Aktivierungskarte (engl. activation map) entspricht einer Faltungsoperation mit einem bestimmten Filter und dem zugehörigen Eingangsbild oder den räumlichen Daten der vorherigen Schicht. In den meisten Fällen werden nicht nur ein, sondern mehrere Filter in einer Faltungsschicht gelernt, so dass es mehrere Aktivierungskarten gibt. In diesem speziellen Fall scheint die Ausgabegröße dieser Faltungsschicht im Vergleich zur Eingabegröße größer geworden zu sein. Infolgedessen werden häufig Pooling-Operationen angehängt, um die Daten innerhalb des Netzwerks zu reduzieren. Die nächste Schicht erhält dann wieder räumliche Informationen und verwendet Filter, um die räumlichen Informationen zu extrahieren und zu verändern.


**Idea**: _`Spärliche Verbindungen (engl. Sparse Connections)` (nicht vollständig verbundene Schichten wie bei einem MLP) sind als Kernel für große Datenstrukturen gegeben. Die Anzahl der lernbaren Gewichte sinkt!_

Vergleichen wir eine standardmäßige voll verbundene Schicht (engl. fully connected layer) eines MLP mit einer Faltungsschicht für ein reguläres farbiges Bild der Größe $256\times256\times3$:
- Erstes Hidden Layer in einer voll verbundenen Schicht:
    - Input Neuronen $\rightarrow$ $256*256*3$
    - Beginnen Sie z. B. mit der Hälfte der Neuronen im ersten Hidden Layer $\rightarrow$ $128*256*3$
    - Ergebnisse in Gewichte und Biases $\rightarrow$ $256*256*3*128*256*3 + 128*256*3 = 19.327.451.136$ Parameters

        
- Erste Faltungsschicht in einem faltigen neuronalen Netz: Standard 256 Filter (vernünftige Größe) der Größe $3\times3\times3$ 
    - Gewichte und Biases $\rightarrow$ $256 * 3 * 3 *3 + 256 = 7.168 $ Parameters
    
Trotzdem brauchen Faltungen mit räumlichen Blöcken wie in der obigen Abbildung noch Zeit, um verarbeitet zu werden.
Lokale Informationen werden nur nicht wie globale Abhängigkeiten in Hidden Layers verwendet!

Die **Vorteile** einer Faltungsschicht (`CONV`) gegenüber einer vollverknüpften Schicht sind die folgenden
 - Weniger Parameter für das Training
 - Nutzung der lokalen Strukturen des Bildes
 - Unabhängig von der Position des Merkmals im Bild
 
**Nachteile** von Faltungsschichten (`CONV`):
 - Informationen müssen räumliche Abhängigkeiten haben (wie bei einem menschlich erkennbaren Bild)

Beim Stapeln mehrerer Faltungsschichten hat ein Kernel der folgenden Faltungsschicht die Form $K_x \times K_y \times d$, wobei $d$ die Anzahl der Kanäle der vorherigen Schicht ist. Die Anzahl der Kanäle ist gegeben durch die Anzahl der verschiedenen Filter, die in der Faltungsschicht verwendet werden. Definiert man also eine Faltungsschicht mit z. B. $nb\_filters=64$, so legt man die dritte Dimension eines Filters in der nächsten Schicht fest. Denn im zweidimensionalen Fall expandiert der Filter immer auf die vorherige Kanaldimension. Betrachtet man CNNs für die Videoanalyse oder für Zeitreihen, so stößt man auf 3-dimensionale Faltungsschichten, die sich nicht nur in den Bilddimensionen bewegen, sondern in einer dritten Dimension (in diesem Fall: Zeit). 

#### Die Poolingsschicht (engl. pooling layer)

<img src="images/maxpool.png" alt="Drawing" style="width: 600px;"/>


                                     Quelle: http://cs231n.github.io/convolutional-networks/


Die Pooling Schicht ist ein Filter wie alle anderen Filter im neuronalen Faltungsnetzwerk. Allerdings mit der Ausnahme, dass sie ihre Gewichte nicht aktualisiert und eine feste Funktionsoperation durchführt. Die häufigste Pooling-Operation ist das Max-Pooling. Wie der Name schon sagt, wird im Bereich des Kerns nur der Maximalwert weitergegeben. Normalerweise entspricht der Stride den Dimensionen des Kernels. Das Max-Pooling wird nur auf die Höhe und Breite des Bildes angewendet, so dass die Kanaldimensionen nicht betroffen sind. Es wird verwendet, um räumliche Informationen zu reduzieren.

<div class="alert alert-block alert-success">
<b>Aufgabe 5.2.2:</b> Implementieren Sie die Funktion <code>max_pool</code> die Maxpooling durchführt. Gegeben ist wieder ein Grauwertbild <code>image_data</code>, d.h. es besitzt nur einen Kanal und Sie können annehmen, dass das Bilder wieder als eine Liste von Listen übergeben wird. Außerdem ist die Größe des Filters <code>filter_size</code> als Tupel und die <code>stride</code> als <code>int</code> gegeben.
</div>

In [16]:
def max_pool(image_data:list, filter_size:tuple, stride:int)->list:
    # STUDENT CODE HERE

    # STUDENT CODE until HERE

In [17]:
test_input_data = [[0,0,0,0,0], [0,1,1,1,0], [0,0,2,0,0], [0,3,3,3,0], [0,0,0,0,0], [0,0,0,0,0]]
test_filter_size = (2,2)
stride = 2
test_result = [[1,1],[3,3], [0,0]]

# The folgende Zeile erzeugt einen Fehler, wenn die Ausgabe der Methode nicht mit der erwarteten übereinstimmt 
found = max_pool(test_input_data, test_filter_size, stride)
assert found == test_result

#### ReLU - Schicht oder Aktivierung
Die "RELU"-Schicht oder Aktivierung verwendet eine elementweise Aktivierungsfunktion auf das Raumvolumen an, wie auf jeden Knoten in einer Hidden Layer. Die Funktion kann als $max(0,x)$ angegeben werden und ist unten dargestellt. Betrachten Sie $\sigma(x)$ als die Aktivierungsfunktion.

In [18]:
import numpy as np
import matplotlib.pyplot as plt

def relu(x:float)->float:
    return np.maximum(0,x)
x = np.linspace(-10, 10, num = 1000)

plt.figure(2, figsize=(10,3.5))
plt.plot(x, relu(x), label='ReLU')
plt.title('The ReLU activation')
plt.legend()
plt.xlabel('x')
plt.ylabel('$\sigma(x)$')
plt.tight_layout()
plt.grid()

### Zusammenfassung

Die folgende Animation zeigt recht gut, wie ein Faltungsnetzwerk (engl. convolutional network) anhand des `MNIST`-Datensatzes funktioniert.
Nachdem die Faltungsschichten die Repräsentation der Bilder verändert haben, werden die endgültigen mehrdimensionalen Blöcke in ein langes Array gelegt (die Operation wird "Flattening" genannt) und an voll verbundene Schichten eines neuronalen Netzes weitergeleitet.

[MNIST-CLassification](http://scs.ryerson.ca/~aharley/vis/conv/flat.html)

#### Receptive Field

In der Animation bzw. Simulation von MNIST werden Abhängigkeiten, die als Linien zwischen mehr als zwei Schichten dargestellt werden, nicht abgebildet.
Dennoch ist es möglich, Beziehungen zwischen beliebigen Schichten innerhalb des Netzes darzustellen. Dadurch ist es möglich, ein gewisses Wissen oder eine Idee über die Anzahl der Faltungsschichten zu erhalten, die für eine Anwendung oder Aufgabe verwendet werden sollten. Betrachten Sie drei übereinander gestapelte Faltungsschichten wie im Bild unten. Ein Wert in der grünen Schicht bezieht sich auf 9 Eingangswerte. Folglich summiert sich ein Wert in der gelben Schicht auf 9 in der grünen Schicht. Ein Eintrag in der gelben Schicht wird also von mehr Werten beeinflusst als die grünen Aktivierungseinträge in Bezug auf das Eingangsbild. Dieser Bereich ist gelb dargestellt und deckt 49 Werte des Eingangsbildes ab. Um die Dimensionen während der Faltungen wie in üblichen CNNs beizubehalten, wurde ein Padding verwendet, um die Dimensionen der Matrix gleich zu halten. Die `Initialmatrix` ist dann von der Größe $7 \times 7$.

<img src="images/ReceptiveField.png" alt="Drawing" style="width: 600px;"/>

    Quelle:https://medium.com/mlreview/a-guide-to-receptive-field-arithmetic-for-convolutional-neural-networks-e0f514068807

<div class="alert alert-block alert-success">
<b>Frage 5.2.3:</b> Was ist der Hauptunterschied zwischen einer Faltungsschicht (engl. convolutional layer) und einer vollverknüpften Schicht (engl. fully-connected layer) und warum werden überhaupt Filter verwendet?
</div>

<div class="alert alert-block alert-success">
<b>Ihre Antwort:</b></div>
